# Analysis on Consumer Preference Data for Black Coffee - Prabakaran Chandran - UChicago - MS ADS

Further Information on Data Available : https://github.com/prabakaranc98/ILoveCoffee/

### Information and Description of Data :
The dataset provided here contains physical, chemical, and consumer preference data for 118 individual consumers who each tasted 27 distinct coffees prepared with precisely controlled brewing conditions, yielding a total of 3,186 individual tastings of black coffee.  Physical measurements include the temperature, total dissolved solids, and percent extraction of the brewed coffee; chemical measurements include the pH and titratable acidity of the brewed coffee; and the consumer preference measurements include hedonic liking, just-about-right assessments of beverage temperature, flavor intensity, mouthfeel, and acidity, and purchase intent.


In [1]:
#importing the neccessary Libraries

import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

from IPython import display

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# read the cotter's coffee preference dataset

coffee_testing = pd.read_csv("/content/cotter_dataset.csv")

coffee_testing.head()

,Judge,Cluster,Week,Session Number,Position,Brew,Temp.x,TDS.x,PE.x,Dose,Setting,Grind,Empty Carafe,Full Carafe,Brew Mass,TDS__1,Percent Extraction,pH,Initial NaOH,Final NaOH,Titration pH,Volume,Brew Temperature,Pour Temp,90Sec Temp,Liking,Temp,Flavor.intensity,Acidity,Mouthfeel,Tea.floral,Fruit,Citrus,Green.veg,Paper.wood,Burnt,Cereal,Nutty,Dark.chocolate,Caramel,Bitter,Astringent,Roasted,Sour,Thick.viscous,Sweet,Rubber,Purchase.intent
0,1,1,3,13,3,87-1.0-16,Low,Low,Low,176.1,1SM,5,1769.3,4598.4,2829.1,1.05,16.868569,5.05,13.3,18.3,8.20,5.0,81.1,67.0,63.5,6,4,2,2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2
1,1,1,2,10,2,87-1.0-20,Low,Low,Medium,145.0,6LG,3,1764.3,4670.2,2905.9,0.97,19.439469,5.01,14.3,18.8,8.20,4.5,81.0,69.4,63.9,4,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1
2,1,1,3,13,9,87-1.0-24,Low,Low,High,119.0,4LG,3,1791.0,4675.1,2884.1,0.96,23.266689,5.11,18.5,22.6,8.20,4.1,78.6,66.3,61.5,2,4,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,2
3,1,1,2,10,9,87-1.25-16,Low,Medium,Low,215.3,12LG,5,1775.1,4586.2,2811.1,1.07,13.970632,5.08,3.6,8.6,8.20,5.0,80.4,68.9,62.1,7,3,3,4,4,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3
4,1,1,3,13,10,87-1.25-20,Low,Medium,Medium,176.1,12LG,3,1782.1,4651.3,2869.2,1.27,20.692129,5.02,7.3,13.1,8.21,5.8,80.8,66.9,62.1,8,3,3,3,3,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,5


In [3]:
# coffee_testing.info() --> as there are major null values
missing_data_columns = coffee_testing.isnull().sum()
null_columns = missing_data_columns[missing_data_columns > 0]

display.display(null_columns)

,0
Titration pH,84


Only the Titration pH is having 84 null value columns , we shall not worry about this as this number is less than 3-5% and also in one particular columnand also , the primary analysis that we are going to focus on is
- How an Individual's decision on preferrence be impacted by various factors staring from relationship between sensory attributes and preference/intent
- How sensory attributes get impacted by the chemical and brewing properties , then focus on the way it is being served / how the diminishing temperature makes the preferrence impacted - I like to drink black coffees to be cooler than piping hot temperature - Let'see

### Irrespective of the brew parameters (if we consider brew and grinding parameters are the instrument variable on the sensory attributes) , the sensory attributes shall show the variation in liking / purchace intent

In [4]:
identifier_variables = ["Judge","Week","Session Number","Brew"]
sensory_attributes = ["Temp","Flavor.intensity","Acidity","Mouthfeel"]

impact_variables = ["Liking","Purchase.intent"]
coffee_testing_sensory = coffee_testing[identifier_variables+sensory_attributes+impact_variables]

In [5]:
### Let's calculate for each attribute variation , how does the liking percentage changes - unweighted ( we are not weighting this based on available samples for each value of the attributes)
def calculate_percentage(df, attribute):
    grouped = df.groupby([attribute, "Liking"]).size().reset_index(name="Count")
    total_counts = grouped.groupby(attribute)["Count"].transform("sum")
    grouped["Percentage"] = (grouped["Count"] / total_counts) * 100
    return grouped

# Prepare data for each sensory attribute
temp_data = calculate_percentage(coffee_testing_sensory, "Temp")
mouthfeel_data = calculate_percentage(coffee_testing_sensory, "Mouthfeel")
flavor_data = calculate_percentage(coffee_testing_sensory, "Flavor.intensity")
acidity_data = calculate_percentage(coffee_testing_sensory, "Acidity")

In [6]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=["Temp vs Liking Distribution", "Mouthfeel vs Liking Distribution",
                    "Flavor Intensity vs Liking Distribution", "Acidity vs Liking Distribution"]
)
color_scale = px.colors.sequential.Viridis

# Temp vs Liking
for idx, liking in enumerate(temp_data["Liking"].unique()):
    temp_subset = temp_data[temp_data["Liking"] == liking]
    fig.add_trace(
        go.Bar(x=temp_subset["Temp"], y=temp_subset["Percentage"],
               name=f"Liking {liking}", legendgroup=f"Liking",
               marker_color=color_scale[idx]),
        row=1, col=1
    )

# Mouthfeel vs Liking
for idx, liking in enumerate(mouthfeel_data["Liking"].unique()):
    mouthfeel_subset = mouthfeel_data[mouthfeel_data["Liking"] == liking]
    fig.add_trace(
        go.Bar(x=mouthfeel_subset["Mouthfeel"], y=mouthfeel_subset["Percentage"],
               name=f"Liking {liking}", legendgroup=f"Liking", showlegend=False,
               marker_color=color_scale[idx]),
        row=1, col=2
    )

# Flavor Intensity vs Liking
for idx, liking in enumerate(flavor_data["Liking"].unique()):
    flavor_subset = flavor_data[flavor_data["Liking"] == liking]
    fig.add_trace(
        go.Bar(x=flavor_subset["Flavor.intensity"], y=flavor_subset["Percentage"],
               name=f"Liking {liking}", legendgroup=f"Liking",
               showlegend=False,
               marker_color=color_scale[idx]),
        row=2, col=1
    )

# Acidity vs Liking
for idx, liking in enumerate(acidity_data["Liking"].unique()):
    acidity_subset = acidity_data[acidity_data["Liking"] == liking]
    fig.add_trace(
        go.Bar(x=acidity_subset["Acidity"], y=acidity_subset["Percentage"],
               name=f"Liking {liking}", legendgroup=f"Liking", showlegend=False,
               marker_color=color_scale[idx]),
        row=2, col=2
    )

fig.update_layout(
    title="Sensory Attributes vs Liking Distribution (Percentages of people voted for specific hedonic score)",
    height=800,
    width=1000,
    barmode="stack",
    showlegend=True,
    xaxis_title="Temp",
    xaxis2_title="Mouthfeel",
    xaxis3_title="Flavor Intensity",
    xaxis4_title="Acidity",
    yaxis_title="Percentage of Liking",
    yaxis2_title="Percentage of Liking",
    yaxis3_title="Percentage of Liking",
    yaxis4_title="Percentage of Liking",
)
fig.show()

### We could see that , Most people like (hedonic scale > 6 ) is very evidentt in the medium range of temerature , internsity,acidity and mouth feel
- very significant percentage for score 8 and 9 we could see in value 3 for all the attributes, 8 and 9 denotes they like the product verymuch to strongly as per hedonic score
- are we in simpson paradox , can we just try to divide this further into the brew condition  and other chemical parameters

as We could see flavouring attributes and NaOH , pH would also impact / as an instrument variable to the sensory attributes - a good causal discovery problem , this analysis let's focus on the asssociation but to reduce the simpson pardoxial condition

the avaialble brewing condition , for example denotes the target brew conditions. The first number identifies the target brew temperature, the second number identified the target total dissolved solids (TDS), and the third number identifies the target percent extraction (PE). For example, 87-1.0-16 denotes 87 degrees Celsius, 1% TDS, and 16% PE

In [7]:
coffee_testing_sensory["Brew"].unique()

array(['87-1.0-16', '87-1.0-20', '87-1.0-24', '87-1.25-16', '87-1.25-20',
       '87-1.25-24', '87-1.5-16', '87-1.5-20', '87-1.5-24', '90-1.0-16',
       '90-1.0-20', '90-1.0-24', '90-1.25-16', '90-1.25-20', '90-1.25-24',
       '90-1.5-16', '90-1.5-20', '90-1.5-24', '93-1.0-16', '93-1.0-20',
       '93-1.0-24', '93-1.25-16', '93-1.25-20', '93-1.25-24', '93-1.5-16',
       '93-1.5-20', '93-1.5-24'], dtype=object)

I am converting the Liking intent of 7,8,9 into Really Liking flag to understand the percentage of people Like the coffee moderate to very much

In [8]:
coffee_testing_sensory["ReallyLiking"] = coffee_testing_sensory.apply(lambda x : 1 if x["Liking"] in [7,8,9] else 0,axis=1)

In [9]:
acid_grouped = coffee_testing_sensory.groupby(["Brew", "Acidity"])["ReallyLiking"].sum().reset_index()

# Step 2: Calculate cumulative percentage of Liking for each Brew
acid_grouped["Total_Liking_Per_Brew"] = acid_grouped.groupby("Brew")["ReallyLiking"].transform("sum")
acid_grouped["Cumulative_Liking"] = acid_grouped.groupby("Brew")["ReallyLiking"].cumsum()
acid_grouped["Cumulative_Percentage"] = (
    acid_grouped["Cumulative_Liking"] / acid_grouped["Total_Liking_Per_Brew"] * 100
)

# Visualize cumulative percentages as a Pareto-like chart , I am Just trying to follow this "elbow curve" analogy , where the sudden increase / more people tend to like
fig = px.line(
    acid_grouped,
    x="Acidity",
    y="Cumulative_Percentage",
    color="Brew",
    markers=True,
    title="Cumulative Liking Percentage by Acidity for Each Brew",
    labels={"Acidity": "Acidity Level", "Cumulative_Percentage": "Cumulative % of ReallyLiking", "Brew": "Brew Style"},
    line_group="Brew",
)
fig.add_hline(y=60, line_dash="dash", annotation_text="60% Threshold", annotation_position="top left")
fig.add_hline(y=80, line_dash="dash", annotation_text="80% Threshold", annotation_position="top left")
fig.add_hline(y=70, line_dash="dash", annotation_text="70% Threshold", annotation_position="top left")

fig.add_hline(y=25, line_dash="dash", annotation_text="25% Percentage of people like ", annotation_position="top left")



# Show plot
fig.show()


If we see , the graph surges from 25% of people liking the coffee from range near 2.25 , to the % of Population really likes become 60-90% when it moves towards the acidity level whent it reaches around range 3 acidity level "just-about-right", further the change is not that great comare to 25% to 80% jump
- and we could see this considently across different brew style , which is more people tend to like moderate acidity in the coffee and we could see the same thing for temp , flavour intensity as well , ( i do next analysis just to show the different storylines)

Let's check What make's thisacidity level "just-about-right" - NaOH and Titration pH Balance?

In [10]:
tit_ph_acid = coffee_testing[["Brew","Titration pH","Acidity"]]
tit_ph_acid.describe()

,Titration pH,Acidity
count,3102.000000,3186.000000
mean,8.200329,3.141871
std,0.010188,0.829192
min,8.180000,1.000000
25%,8.190000,3.000000
50%,8.200000,3.000000
75%,8.210000,4.000000
max,8.230000,5.000000


In [11]:
fig = px.scatter(
    tit_ph_acid,
    x="Titration pH",
    y="Acidity",
    trendline="lowess",  # Adds LOESS smoothing
    title="LOESS Smoothing of Titration pH vs Acidity by Brew",
    labels={"Titration pH": "Titration pH", "Acidity": "Acidity", "Brew": "Brew Style"}
)

# Show plot
fig.show()

could not see a pattern / variation with respect to the Titration pH to Judgement of Acidity - so it naturally asks the question when acidity is not right , let's say too little or too high? - Is it temperature make it "feel the acidity" in different ways? as the Acidity score are given by the testers based on what they feel

##### 90Sec Temperature: the temperature, in degrees Celsius, of the brew in the paper cup after a 90-second wait to allow cooling.

# Let's consider 90Sec temparature and Acidity JAR and 1 and 5 extreme conditions - if I form the conditional probability , p(JAR | temperature in speciifci value)

In [12]:
temp_testing = coffee_testing[["90Sec Temp","Acidity"]]
## Define smaller bins for 90Sec Temp - we can do this analysis for different temperatuee as well , I assume brew temp play major role in acidity

bin_edges = np.histogram_bin_edges(temp_testing["90Sec Temp"], bins=25)
temp_testing["Temp_Bin"] = pd.cut(temp_testing["90Sec Temp"], bins=bin_edges)

acidity_levels = [1, 3, 5]
conditional_probabilities = pd.DataFrame()

#Calculate conditional probabilities for each Acidity level
for acidity in acidity_levels:
    filtered_df = temp_testing[temp_testing["Acidity"] == acidity]
    prob_df = filtered_df.groupby("Temp_Bin").size().reset_index(name="Count")
    total_counts = temp_testing.groupby("Temp_Bin").size()

    #calculate probabilities
    prob_df = prob_df.set_index("Temp_Bin").reindex(total_counts.index, fill_value=0).reset_index()
    prob_df["Probability"] = prob_df["Count"] / total_counts.values
    prob_df["Acidity"] = acidity
    conditional_probabilities = pd.concat([conditional_probabilities, prob_df], ignore_index=True)

conditional_probabilities["Temp_Bin_Str"] = conditional_probabilities["Temp_Bin"].astype(str)

fig = px.line(
    conditional_probabilities,
    x="Temp_Bin_Str",
    y="Probability",
    color="Acidity",
    title="Conditional Probability Across Temperature Bins for Acidity Levels",
    labels={
        "Temp_Bin_Str": "90Sec Temp Bin",
        "Probability": "Conditional Probability",
        "Acidity": "Acidity Level"
    },
    markers=True
)

fig.update_layout(
    xaxis=dict(tickangle=90),  # Rotate x-axis labels
    yaxis_title="Conditional Probability",
    xaxis_title="90Sec Temp Bin"
)

fig.show()

## Note : when i do this for brew and pour temperature , the conditional probablity patterns dont differ , which states that , most of the time irrespective
## of the temperature, the acidity levels are just right. at level 3

In [13]:
### this analysis can further proceed to find the real confounder ! and expand to other variable relationship , I have tried to showcase my thoughtproicess